In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,first_value,count
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

job_skills = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/online") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "job_skills") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [7]:
job_skills\
    .withColumn("cnt",count("job_role").over(Window.orderBy(col("row_id"))))\
    .withColumn("job_role_main",first_value("job_role").over(Window.partitionBy("cnt").orderBy(col("cnt"))))\
    .drop("job_role")\
    .withColumnRenamed("job_role_main","job_role")\
    .select("row_id","job_role","skills")\
    .show()

24/03/21 20:28:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/21 20:28:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/21 20:28:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/21 20:28:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+--------------+----------------+
|row_id|      job_role|          skills|
+------+--------------+----------------+
|     1| Data Engineer|             SQL|
|     2| Data Engineer|          Python|
|     3| Data Engineer|             AWS|
|     4| Data Engineer|       Snowflake|
|     5| Data Engineer|    Apache Spark|
|     6| Web Developer|            Java|
|     7| Web Developer|            HTML|
|     8| Web Developer|             CSS|
|     9|Data Scientist|          Python|
|    10|Data Scientist|Machine Learning|
|    11|Data Scientist|   Deep Learning|
|    12|Data Scientist|         Tableau|
+------+--------------+----------------+

